In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pykeen
import torch
from pykeen.pipeline import pipeline

In [7]:
dataset = 'WN18RR'
num_epochs = 10
embedding_dim = 20
loss = 'BCEWithLogitsLoss'
margin = 1.0

In [8]:
ds = pykeen.datasets.get_dataset(dataset=dataset)

training = ds.training.mapped_triples
ds.training.mapped_triples = training[training[:,0].sort()[1]]
testing = ds.testing.mapped_triples
ds.testing.mapped_triples = testing[testing[:,0].sort()[1]]
validation = ds.validation.mapped_triples
ds.validation.mapped_triples = validation[validation[:,0].sort()[1]]

In [9]:
ds.training.mapped_triples[:20]

tensor([[    0,     0,    20],
        [    0,     0,    15],
        [    0,    10,     5],
        [    0,    10,     3],
        [    0,     1, 21500],
        [    0,     1, 18864],
        [    0,     1, 20627],
        [    0,     1,  4521],
        [    0,     1, 18583],
        [    1,     3,     0],
        [    2,     3,     0],
        [    2,     1,  3761],
        [    3,     1, 16759],
        [    3,     1,  4521],
        [    3,     3, 11889],
        [    4,     3,     1],
        [    5,     3,     0],
        [    5,    10,     0],
        [    5,     1, 16759],
        [    6,     1,  2901]])

In [10]:
import functools
import itertools
from typing import Optional

from pykeen.models import StructuredEmbedding
from pykeen.models.base import EntityEmbeddingModel
from pykeen.nn import Embedding
from pykeen.losses import Loss
from pykeen.nn.init import xavier_uniform_
from pykeen.regularizers import Regularizer
from pykeen.triples import TriplesFactory
from pykeen.typing import DeviceHint
from pykeen.utils import compose

class ModifiedSE(StructuredEmbedding):
    
    def __init__(
        self,
        triples_factory: TriplesFactory,
        embedding_dim: int = 50,
        scoring_fct_norm: int = 2,
        loss: Optional[Loss] = None,
        preferred_device: DeviceHint = None,
        random_seed: Optional[int] = None,
        regularizer: Optional[Regularizer] = None,
    ) -> None:
        r"""Initialize SE.

        :param embedding_dim: The entity embedding dimension $d$. Is usually $d \in [50, 300]$.
        :param scoring_fct_norm: The $l_p$ norm. Usually 1 for SE.
        """
        super().__init__(
            triples_factory=triples_factory,
            embedding_dim=embedding_dim,
            loss=loss,
            preferred_device=preferred_device,
            random_seed=random_seed,
            regularizer=regularizer,
        )
            
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:  # noqa: D102
        # Get embeddings
        h = self.entity_embeddings(indices=hrt_batch[:, 0]).view(-1, self.embedding_dim, 1)
        rel_h = self.left_relation_embeddings(indices=hrt_batch[:, 1]).view(-1, self.embedding_dim, self.embedding_dim)
        rel_t = self.right_relation_embeddings(indices=hrt_batch[:, 1]).view(-1, self.embedding_dim, self.embedding_dim)
        t = self.entity_embeddings(indices=hrt_batch[:, 2]).view(-1, self.embedding_dim, 1)
        
        # Project entities
        proj_h = rel_h @ h
        proj_t = rel_t @ t
        
        norms = torch.norm(proj_h - proj_t, dim=1, p=self.scoring_fct_norm)**2
        scores = -norms
        return scores
    
    def score_t(self, hr_batch: torch.LongTensor, slice_size: int = None) -> torch.FloatTensor:  # noqa: D102
        # Get embeddings
        h = self.entity_embeddings(indices=hr_batch[:, 0]).view(-1, self.embedding_dim, 1)
        rel_h = self.left_relation_embeddings(indices=hr_batch[:, 1]).view(-1, self.embedding_dim, self.embedding_dim)
        rel_t = self.right_relation_embeddings(indices=hr_batch[:, 1])
        rel_t = rel_t.view(-1, 1, self.embedding_dim, self.embedding_dim)
        t_all = self.entity_embeddings(indices=None).view(1, -1, self.embedding_dim, 1)

        if slice_size is not None:
            proj_t_arr = []
            # Project entities
            proj_h = rel_h @ h

            for t in torch.split(t_all, slice_size, dim=1):
                # Project entities
                proj_t = rel_t @ t
                proj_t_arr.append(proj_t)

            proj_t = torch.cat(proj_t_arr, dim=1)

        else:
            # Project entities
            proj_h = rel_h @ h
            proj_t = rel_t @ t_all
        norms = torch.norm(proj_h[:, None, :, 0] - proj_t[:, :, :, 0], dim=-1, p=self.scoring_fct_norm)**2
        scores = -norms
        return scores


    def score_h(self, rt_batch: torch.LongTensor, slice_size: int = None) -> torch.FloatTensor:  # noqa: D102
        # Get embeddings
        h_all = self.entity_embeddings(indices=None).view(1, -1, self.embedding_dim, 1)
        rel_h = self.left_relation_embeddings(indices=rt_batch[:, 0])
        rel_h = rel_h.view(-1, 1, self.embedding_dim, self.embedding_dim)
        rel_t = self.right_relation_embeddings(indices=rt_batch[:, 0]).view(-1, self.embedding_dim, self.embedding_dim)
        t = self.entity_embeddings(indices=rt_batch[:, 1]).view(-1, self.embedding_dim, 1)

        if slice_size is not None:
            proj_h_arr = []

            # Project entities
            proj_t = rel_t @ t

            for h in torch.split(h_all, slice_size, dim=1):
                # Project entities
                proj_h = rel_h @ h
                proj_h_arr.append(proj_h)

            proj_h = torch.cat(proj_h_arr, dim=1)
        else:
            # Project entities
            proj_h = rel_h @ h_all
            proj_t = rel_t @ t

        norms = torch.norm(proj_h[:, :, :, 0] - proj_t[:, None, :, 0], dim=-1, p=self.scoring_fct_norm)**2
        print(norms.shape)
        scores = -norms
        return scores

In [11]:
result2 = pipeline(
    model=ModifiedSE,
    dataset=dataset,
    random_seed=1235,
    device='gpu',
    training_kwargs=dict(num_epochs=num_epochs, sampler='presorted'),
    model_kwargs=dict(embedding_dim=embedding_dim),
    loss=loss,
    loss_kwargs=dict()
)
model2 = result2.model
model2

Not Shuffling!


INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...



torch.Size([64, 40559])
torch.Size([64, 40559])


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=64.


torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])
torch.Size([64, 40559])



KeyboardInterrupt: 

In [ ]:
result2.plot_losses()
plt.show()

In [ ]:
comp_models = ['StructuredEmbedding','TransE','RotatE','HolE']
comp_results = []
for comp_model in comp_models:
    print('Running {}'.format(comp_model))
    result = pipeline(
        dataset=dataset,
        model=comp_model,
        random_seed=1235,
        device='gpu',
        training_kwargs=dict(num_epochs=num_epochs),
        model_kwargs=dict(embedding_dim=embedding_dim),
        loss=loss,
        loss_kwargs=dict()
    )
    comp_results.append(result)

In [ ]:
plt.plot(np.arange(len(result2.losses)),result2.losses,label='Sheaf SE')
for i in range(len(comp_models)):
    comp_model = comp_models[i]
    comp_result = comp_results[i]
    plt.plot(np.arange(len(comp_result.losses)),comp_result.losses,label=comp_model)
plt.ylabel(str(result.model.loss).replace('()',''))
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
res_df = result2.metric_results.to_df()

In [ ]:
compto = 1

In [ ]:
res_df['diff'] = res_df.Value - comp_results[compto].metric_results.to_df().Value
res_df

In [ ]:
# comp_results[0].model.score_all_triples()

In [ ]:
# model2.score_all_triples()